In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:

# Carregar os ficheiros
voos = pd.read_csv("viagens/flights_ratings_modelos.csv")
aeroportos_partida = pd.read_csv("airport/airports.csv")
aeroportos_chegada = pd.read_csv("airport/airports.csv")
aeronaves = pd.read_csv("airplane/aeronaves.csv")
companhias = pd.read_csv("airline/airlines.csv")

# 2. Renomear colunas relevantes
aeronaves.rename(columns={"Modelo": "Modelo Avião"}, inplace=True)
companhias.rename(columns={"Name": "nome_companhia_aerea", "IATA": "id_companhia_aerea", "Country" : "pais_companhia_aerea"}, inplace=True)
aeroportos_partida.rename(columns={"Nome": "nome_aeroporto_partida", "Pais": "pais_aeroporto_partida", "Cidade" : "Cidade_partida", "Latitude" : "Latitude_partida", "Longitude" : "Longitude_partida", "IATA" : "id_aeroporto_partida"}, inplace=True)
aeroportos_chegada.rename(columns={"Nome": "nome_aeroporto_chegada", "Pais": "pais_aeroporto_chegada", "Cidade" : "Cidade_chegada", "Latitude" : "Latitude_chegada", "Longitude" : "Longitude_chegada", "IATA" : "id_aeroporto_chegada"}, inplace=True)

# 3. Juntar os dados
voos = voos.merge(aeronaves, on="Modelo Avião", how="left")
voos = voos.merge(
    companhias[["id_companhia_aerea", "nome_companhia_aerea", "pais_companhia_aerea"]], 
    left_on="AIRLINE_CODE",
    right_on="id_companhia_aerea",
    how="left")
voos = voos.merge(
    aeroportos_partida[["id_aeroporto_partida", "nome_aeroporto_partida", "pais_aeroporto_partida", "Cidade_partida", "Latitude_partida", "Longitude_partida"]], 
    left_on="ORIGIN", 
    right_on="id_aeroporto_partida", 
    how="left")
voos = voos.merge(
    aeroportos_chegada[["id_aeroporto_chegada", "nome_aeroporto_chegada", "pais_aeroporto_chegada", "Cidade_chegada", "Latitude_chegada", "Longitude_chegada"]], 
    left_on="DEST", 
    right_on="id_aeroporto_chegada", 
    how="left")

In [ ]:
# 4. Preparar dados finais
voos_final = pd.DataFrame({
    "nr_voo": voos["FL_NUMBER"],
    "data_partida": voos["FL_DATE"],
    "nome_companhia_aerea": voos["nome_companhia_aerea"],
    "id_companhia_aerea": voos["id_companhia_aerea"],
    "pais_companhia_aerea": voos["pais_companhia_aerea"],
    "aeroporto_origem": voos["id_aeroporto_partida"],
    "nome_aeroporto_origem": voos["nome_aeroporto_partida"],
    "pais_aeroporto_origem": voos["pais_aeroporto_partida"],
    "cidade_aeroporto_origem": voos["Cidade_partida"],
    "latitude_aeroporto_origem": voos["Latitude_partida"],
    "longitude_aeroporto_origem": voos["Longitude_partida"],
    "aeroporto_destino": voos["id_aeroporto_chegada"],
    "nome_aeroporto_destino": voos["nome_aeroporto_chegada"],
    "pais_aeroporto_destino": voos["pais_aeroporto_chegada"],
    "cidade_aeroporto_destino": voos["Cidade_chegada"],
    "latitude_aeroporto_destino": voos["Latitude_chegada"],
    "longitude_aeroporto_destino": voos["Longitude_chegada"],
    "hora_partida_real": voos["DEP_TIME"],
    "hora_partida_esperada": voos["CRS_DEP_TIME"],
    "hora_chegada_real": voos["ARR_TIME"],
    "hora_chegada_esperada": voos["CRS_ARR_TIME"],
    "cancelado": voos["CANCELLED"],
    "desviado": voos["DIVERTED"],
    "distancia": voos["DISTANCE"],
    "atraso_companhia_aerea": voos["DELAY_DUE_CARRIER"],
    "atraso_metereologia": voos["DELAY_DUE_WEATHER"],
    "atraso_SNA": voos["DELAY_DUE_NAS"],
    "atraso_seguranca": voos["DELAY_DUE_SECURITY"],
    "atraso_voo_anterior": voos["DELAY_DUE_LATE_AIRCRAFT"],
    "Modelo_Avião": voos["Modelo Avião"],
    "Fabricante": voos["Fabricante"],
    "avaliacao": voos["Rating"]
})

In [ ]:
# 5. Gerar campos climáticos com base nos atrasos
def simular_clima(row):
    clima = {
        "temperatura": np.random.uniform(15, 25),
        "precipitacao": np.random.uniform(0, 5),
        "visibilidade": np.random.uniform(8, 15),
        "cobertura_nuvens": np.random.uniform(0, 50),
        "velocidade": np.random.uniform(5, 20),
        "direcao": np.random.uniform(0, 360)
    }

    if row["atraso_metereologia"] > 0:
        clima["temperatura"] = np.random.uniform(25, 40)
        clima["precipitacao"] = np.random.uniform(5, 50)
        clima["cobertura_nuvens"] = np.random.uniform(60, 100)

    if row["atraso_companhia_aerea"] > 0:
        clima["precipitacao"] += np.random.uniform(3, 20)

    if row["atraso_SNA"] > 0:
        clima["velocidade"] = np.random.uniform(20, 60)
        clima["direcao"] = np.random.uniform(0, 360)

    if row["atraso_seguranca"] > 0:
        clima["cobertura_nuvens"] = max(clima["cobertura_nuvens"], np.random.uniform(70, 100))
        clima["visibilidade"] = np.random.uniform(0.5, 5)

    if row["atraso_voo_anterior"] > 0:
        for key in clima:
            clima[key] *= np.random.uniform(1.05, 1.15)

    return pd.Series(clima)

# 6. Aplicar simulação
clima_df = voos_final.apply(simular_clima, axis=1)
voos_completos = pd.concat([voos_final, clima_df], axis=1)

# 7. Exportar para CSV
voos_completos.to_csv("voos_completos_clima.csv", index=False)

print("✅ Ficheiro 'voos_completos_clima.csv' criado com sucesso.")